In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv
from urllib.request import urlopen
import pandas as pd
import urllib.request

driver = webdriver.Chrome('/chromedriver/chromedriver.exe')
url = "https://www.instagram.com/"

USERID = input("아이디를 입력해주세요: ")
USERPW = input("비밀번호를 입력해주세요: ")

driver.implicitly_wait(10)

driver.get(url)

driver.find_element_by_name("username").click()
driver.find_element_by_name("username").send_keys(USERID)
driver.find_element_by_name("password").click()
driver.find_element_by_name("password").send_keys(USERPW)    
driver.find_elements_by_css_selector(".sqdOP.L3NKy.y3zKF")[1].click()
driver.find_element_by_css_selector(".aOOlW.HoLwm").click()
driver.find_elements_by_css_selector(".gmFkV")[0].click()

SCROLL_PAUSE_TIME = 1.0
reallink = []

while True:
    pageString = driver.page_source
    bsObj = BeautifulSoup(pageString, "lxml")

    for link1 in bsObj.find_all(name="div", attrs={"class" : "Nnq7C weEfm"}):
        title = link1.select('a')[0]
        real = title.attrs['href']
        reallink.append(real)
        title = link1.select('a')[1]
        real = title.attrs['href']
        reallink.append(real)
        title = link1.select('a')[2]
        real = title.attrs['href']
        reallink.append(real)

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

        else:
            last_height = new_height
            continue


csvtext = []

for i in range(len(reallink)):
    csvtext.append([])
    req = "https://www.instagram.com/{0}{1}".format(USERID,reallink[i])

    webpage = urlopen(req)
    soup = BeautifulSoup(webpage, "lxml", from_encoding='utf-8')
    soup1 = soup.find("meta", attrs = {"property":"og:description"})

    reallink1 = soup1['content']
    reallink1 = reallink1[reallink1.find("@")+1:reallink1.find(")")]
    reallink1 = reallink1[:20]
    if reallink1 == "":
        reallink1 = 'Null'
    csvtext[i].append(reallink1)

    for reallink2 in soup.find_all("meta", attrs={"property": "instapp:hashtags"}):
        reallink2 = reallink2['content']
        csvtext[i].append(reallink2)

data= pd.DataFrame(csvtext)
data.to_csv('insta.csv', encoding = 'utf-8')

f = open("insta.csv", encoding="utf-8")
contents = f.readlines()
abc=[]
hashtag = []
for line in contents:
    abc.append(line.strip().split(','))



for i in abc:
    if "여행" in i:
        hashtag.append(i[0])
    else:
        continue



t_link = []
for i in hashtag:
    t_link.append(reallink[int(i)])


loc=[]

for k in t_link:
    o_link = "https://www.instagram.com/{0}{1}".format(USERID,k)
    html = urlopen(o_link)  
    data = BeautifulSoup(html, "html.parser") 
    data = str(data)
    pas = '"@type":"Place","name":"'
    pas_back = '"'
    pas_num = data.find(pas) + len(pas)
    back_num = data[pas_num:].find(pas_back)
    loc.append(data[pas_num:pas_num+back_num].strip().split(',')[0])
loc2=[]
for i in loc:
    if i not in loc2:
        loc2.append(i)
    else:
        continue
loc=loc2
print(loc)

tour_file = open('tour.csv')
tour_contents = tour_file.readlines()
tour_file.close()
tour_list=[]
for i in tour_contents:
    tour_list.append(i)

tour_file = open('tour.csv')
tour_contents = tour_file.readlines()
tour_file.close()
tour_list=[]
loc3=[]
for i in tour_contents:
    tour_list.append(i.strip().replace('\n','').split(','))
for i in tour_list:
    if i[0] in loc:
        loc3.append(i)
        
tour_file = open('tour.csv')
tour_contents = tour_file.readlines()
tour_file.close()
tour_list=[]
loc3=[]
for i in tour_contents:
    tour_list.append(i.strip().replace('\n','').split(','))
for i in tour_list:
    if i[0] in loc:
        loc3.append(i)
print(loc3)

import folium
import base64
m=folium.Map(location=[0,0],zoom_start = 1.5)

color=['red','blue','green','purple','pink','orange','cadetblue','darkpurple','black']

for i,j in zip(loc3,color):
    folium.Marker([i[2],i[1]], icon=folium.Icon(icon='plane', color = j), tooltip = '<i>{}<\i>'.format(i[0]), popup='image').add_to(m)
m

아이디를 입력해주세요: 1999._.sangjin
비밀번호를 입력해주세요: gus153
[]


FileNotFoundError: [Errno 2] No such file or directory: 'tour.csv'